In [1]:
import math

In [2]:
import cv2
import numpy as np

In [14]:
import torch
from torch.utils.serialization import load_lua
from torch.legacy import nn

In [4]:
def getNameList(filePath):
    names = []
    with open(filePath) as f:
        names = [ line.strip() for line in f ]
    return names

In [5]:
inputImg = cv2.imread("datasets/Experiment_1/Aamir_Khan.png")

In [6]:
# re-scale the smaller dim (among width, height) to refSize

refSize, targetSize = 256, 224
imgRows, imgCols = inputImg.shape[0], inputImg.shape[1]
if imgCols < imgRows: resizedImg = cv2.resize(inputImg, (refSize, refSize * imgRows / imgCols))
else: resizedImg = cv2.resize(inputImg, (refSize * imgCols / imgRows, refSize))

In [7]:
# center-crop

oH, oW = targetSize, targetSize
iH, iW = resizedImg.shape[0], resizedImg.shape[1]
anchorH, anchorW = int(math.ceil((iH - oH)/2)), int(math.ceil((iW - oW) / 2))
croppedImg = resizedImg[anchorH:anchorH+oH, anchorW:anchorW+oW]

In [8]:
# convert shape from (height, width, 3) to (3, width, height)

channel_1, channel_2, channel_3 = croppedImg[:, :, 0], croppedImg[:, :, 1], croppedImg[:, :, 2]
croppedImg = np.empty([3, croppedImg.shape[0], croppedImg.shape[1]])
croppedImg[0], croppedImg[1], croppedImg[2] = channel_1, channel_2, channel_3

In [9]:
# subtract training mean

inputImg = inputImg.astype(float)
trainingMean = [129.1863, 104.7624, 93.5940]
for i in range(3): croppedImg[i] = croppedImg[i] - trainingMean[i]

In [10]:
# load pre-trained VGG-Face network and list of IDs

vggFace = load_lua("preTrainedNets/VGG_FACE_pyTorch_small.t7")
vggFace.modules[31] = nn.View(1, 25088)
idNames = getNameList("preTrainedNets/names.txt")

In [11]:
# forward pass

input = torch.Tensor(croppedImg).unsqueeze(0)
output = vggFace.forward(input)
output = output.numpy()

In [12]:
print "ID = ", idNames[np.argmax(output)]

ID =  Aamir_Khan


In [15]:
# top-k predictions

ind = output[0].argsort()[-5:][::-1]
for idx in ind:
    print idNames[idx]

Aamir_Khan
Manish_Dayal
Adam_Driver
John_Abraham
Adam_Copeland
